In [1]:
import math
import os.path as path
import numpy as np
import pandas as pd
from skimage.util.montage import montage2d
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D, BatchNormalization, Dropout, MaxPooling2D, Dense, Flatten, ZeroPadding2D, Activation
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD, Adam
from keras.regularizers import l2
from extra_functions import *


def load_and_format(in_path):
    out_df = pd.read_json(in_path)
    out_images = out_df.apply(lambda c_row: [np.stack([c_row['band_1'],c_row['band_2']], -1).reshape((75,75,2))],1)
    out_images = np.stack(out_images).squeeze()
    return out_df, out_images

dir_path = path.abspath(path.join('__file__',"../.."))
train_path = dir_path + "/train.json"
test_path = dir_path + "/test.json"
train_df, train_images = load_and_format(train_path)
test_df, test_images = load_and_format(test_path)


x_train, x_val, y_train, y_val = train_test_split(train_images,
                                                   to_categorical(train_df['is_iceberg']),
                                                    random_state = 2017,
                                                    test_size = 0.3
                                                   )
print('Train', x_train.shape, y_train.shape)
print('Validation', x_val.shape, y_val.shape)

"""
print("reading data")
x_train = pd.read_csv('x_train.csv', header=None)
x_train = x_train.values
y_train = pd.read_csv('y_train.csv', header=None)
y_train = y_train.values
x_val = pd.read_csv('x_val.csv', header=None)
x_val = x_val.values
y_val = pd.read_csv('y_val.csv', header=None)
y_val = y_val.values  

print("reshaping data")
x_train = np.reshape(x_train, (3366, 75, 75, 2))
x_val = np.reshape(x_val, (482, 75, 75, 2)) """

print('Train', x_train.shape, y_train.shape)
print('Validation', x_val.shape, y_val.shape) 

cnn = Sequential()
#Original: 0.0005
#0.001 -- Pretty good, better than original
#0.01 -- Pretty good, less consistent than above but higher peak
#0.1 -- Very good, peak of 0.8859, quick rise as well, decently consistent
#0.5 -- Very good, 0.89 at epoch 127, peak of 0.8963 at epoch 225, slower to rise, ok consistent
#0.25 -- Very good, 100 epochs has 0.9004, ok consistent
#0.25 w/128 batch -- Very good, 95 epochs has 0.9004, more consistent stretches, slower to rise

#0.01 -- pretty good -- 0.88, pretty consistent
#0.1 -- pretty good -- 0.9, ok consistent

weight_decay = 0.01

#Preprocess Data with Mean Normalization
cnn.add(BatchNormalization(input_shape=(75, 75, 2)))

#Adding Layers
print("Adding CONV Layers")
cnn.add(Conv2D(16, kernel_size=(3,3), activation='relu', kernel_regularizer=l2(weight_decay)))
#cnn.add(BatchNormalization())


#cnn.add(Conv2D(16, kernel_size=(3,3), padding = 'same', activation='relu'))
#cnn.add(BatchNormalization())
cnn.add(MaxPooling2D(pool_size=(2,2)))


#cnn.add(Conv2D(32, kernel_size=(3,3), padding = 'same', activation='relu'))
#cnn.add(BatchNormalization())
#cnn.add(Dropout(0.3))

cnn.add(Conv2D(32, kernel_size=(3,3), activation='relu', kernel_regularizer=l2(weight_decay)))
#cnn.add(BatchNormalization())
cnn.add(MaxPooling2D(pool_size=(2,2)))


cnn.add(Conv2D(48, kernel_size=(3,3), activation='relu', kernel_regularizer=l2(weight_decay)))
#cnn.add(BatchNormalization())

#cnn.add(Conv2D(48, kernel_size=(3,3), padding = 'same', activation='relu'))
#cnn.add(BatchNormalization())

#cnn.add(Conv2D(48, kernel_size=(3,3), padding = 'same', activation='relu', kernel_regularizer=l2(weight_decay)))
#cnn.add(BatchNormalization())
#cnn.add(Dropout(0.3))
cnn.add(MaxPooling2D(pool_size=(2,2)))


#Add Fully-Connected Layer
print("Adding Fully-Connected Layer")
#Flatten so that the data can pass through a FC Layer
cnn.add(Flatten())
cnn.add(Dense(100, activation='relu', kernel_regularizer=l2(weight_decay)))
#cnn.add(BatchNormalization())
cnn.add(Dropout(0.3))

#Add Output Layer
print("Adding Output Layer")
cnn.add(Dense(2, activation='softmax'))

#Define loss function and optimizer
cnn.compile(optimizer='adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
cnn.summary()
print("Training")
cnn.fit(x_train, y_train, batch_size = 64, validation_data = (x_val, y_val), epochs = 100, shuffle = True)

"""
print("predicting")
test_predictions = cnn.predict(test_images)

pred_df = test_df[['id']].copy()
pred_df['is_iceberg'] = test_predictions[:,1]
print("creating csv")
pred_df.to_csv('predictions.csv', index = False) """

Using TensorFlow backend.


Train (1122, 75, 75, 2) (1122, 2)
Validation (482, 75, 75, 2) (482, 2)
Train (1122, 75, 75, 2) (1122, 2)
Validation (482, 75, 75, 2) (482, 2)
Adding CONV Layers
Adding Fully-Connected Layer
Adding Output Layer
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_1 (Batch (None, 75, 75, 2)         8         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 73, 73, 16)        304       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 34, 34, 32)        4640      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 17, 17, 32)        0         
________________________________________________________________

'\nprint("predicting")\ntest_predictions = cnn.predict(test_images)\n\npred_df = test_df[[\'id\']].copy()\npred_df[\'is_iceberg\'] = test_predictions[:,1]\nprint("creating csv")\npred_df.to_csv(\'predictions.csv\', index = False) '